In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('threads/*.json')
len(files)

48167

In [3]:
!mkdir pages

In [6]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}/page/{i}')

100%|███████████████████████████████████| 48167/48167 [00:16<00:00, 2898.49it/s]


In [7]:
urls = sorted(list(set(urls)))

In [8]:
len(urls)

286043

In [9]:
def get_href(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    
    div = soup.find_all('div', {'data-role': "commentContent"})
        
    return [str(d) for d in div]

In [ ]:
max_worker = 2

for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
        
    topics = [url for url in urls[i: i + max_worker]]
    
    data = []    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href, url): url for url in topics}

        for future in as_completed(futures):
            try:
                data.append(future.result())
            except:
                pass
                
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)
        
    try:
        del data
        del futures
    except:
        pass

  2%|▊                               | 3530/143022 [1:01:57<41:38:59,  1.07s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 13%|███▉                           | 18092/143022 [5:25:19<36:49:24,  1.06s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 17%|█████▎                         | 24433/143022 [7:37:45<37:26:38,  1.14s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

In [20]:
files = glob('threads/*.json')
len(files)

48167

In [21]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|████████████████████████████████████| 48167/48167 [06:53<00:00, 116.43it/s]


In [22]:
files = glob('pages/*.json')
len(files)

143022

In [23]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(BeautifulSoup(d_, "lxml").text)

 58%|████████████████████▉               | 82993/143022 [23:17<53:16, 18.78it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
texts = list(set(texts))
len(texts)

5857563

In [25]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|█████████████████████████████| 5857563/5857563 [00:08<00:00, 679402.62it/s]


In [26]:
!ls

everything.jsonl  get-forums.ipynb  get-threads.ipynb  threads
forums		  get-pages.ipynb   pages


In [28]:
# !zip -r pages.zip pages
# !zip -r threads.zip threads
# !zip -r forums.zip forums